In [737]:
from datetime import datetime
import sys
import os
import math
import numpy as np
import pandas as pd
import pandas_ta as ta
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.trend import STCIndicator
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
import matplotlib.dates as mpl_dates

In [738]:
def load_data_file(symbol):
    #  Create output file name
    daily_file_name = "{}_1d_1m.csv".format(symbol)
    daily_full_path = os.path.join('C:\\dev\\trading\\tradesystem1\\data\\crypto\\', daily_file_name)

    #  Check if output file exists
    if os.path.exists(daily_full_path):
        df = pd.read_csv(daily_full_path, parse_dates=True)
        if df.empty:
            return None
        df['Date'] = pd.to_datetime(df['Datetime'])
        df['Date'] = df['Date'].apply(mpl_dates.date2num)
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]
        return df
    else:
        return None

In [739]:
def calculate_tis(df):
    bb_window = 20
    indicator_bb = BollingerBands(close=df['Close'], window=bb_window, window_dev=2)

    # Add Bollinger Bands features
    df['BB_mid'] = indicator_bb.bollinger_mavg()
    df['BB_high'] = indicator_bb.bollinger_hband()
    df['BB_low'] = indicator_bb.bollinger_lband()
        
    rsi_window = 14
    df['RSI'] = ta.rsi(df['Close'], window=rsi_window)
    
    stc_window_slow = 50
    stc_window_fast = 23
    stc_cycle = 10
    indicator_stc = STCIndicator(close=df['Close'], window_slow=stc_window_slow, window_fast=stc_window_fast, cycle=stc_cycle, smooth1=3, smooth2=3)

    # Add features
    df['STC'] = indicator_stc.stc()
    
    
    return df

In [740]:
def calculate_signals(df):
    df['RSI_entry_ind'] = np.where(np.logical_and((df['RSI'] > 34), (df['RSI'].shift() <= 34)), 1, 0)
    df['RSI_exit_ind'] = np.where(np.logical_and((df['RSI'] < 84), (df['RSI'].shift() >= 84)), 1, 0)
    
    #  Calculate upper / lower boundary for BB
    close_prices = df['Close'].to_numpy()
    max_close = np.amax(close_prices)
    min_close = np.amin(close_prices)
    diff_close = max_close - min_close
    df['BB_low_adj'] = df["BB_low"] + (diff_close * 0.09)
    df['BB_entry_ind'] = np.where((df["Close"] <= df["BB_low_adj"]), 1, 0)
    df['BB_high_adj'] = df["BB_high"] - (diff_close * 0.07)
    df['BB_exit_ind'] = np.where((df["Close"] >= df["BB_high_adj"]), 1, 0)
    
    df['STC_entry_ind'] = np.where(np.logical_and(df['STC'] > 73, df['STC'].shift() <= 73), 1, 0)
    df['STC_exit_ind'] = np.where(np.logical_and(df['STC'] < 97, df['STC'].shift() >= 97), 1, 0)
    return df

In [741]:
def execute_strategy(df):
    close_prices = df['Close'].to_numpy()
    rsi_entry = df['RSI_entry_ind'].to_numpy()
    rsi_exit = df['RSI_exit_ind'].to_numpy()  
    bb_entry = df['BB_entry_ind'].to_numpy()
    bb_exit = df['BB_exit_ind'].to_numpy()
    stc_entry = df['STC_entry_ind'].to_numpy()
    stc_exit = df['STC_exit_ind'].to_numpy()  

    required_entry_signals = 3
    required_exit_signals = 3

    last_entry_price = 0
    entry_prices = []
    exit_prices = []
    hold = 0

    for i in range(len(close_prices)):
        current_price = close_prices[i]
        num_entry_signals = 0
        num_exit_signals = 0

        lookback_ind = i - 20
        if lookback_ind >= 0:
            rsi_entry_lookback = rsi_entry[lookback_ind:i]
            if 1 in rsi_entry_lookback:
                num_entry_signals += 1
            rsi_exit_lookback = rsi_exit[lookback_ind:i]
            if 1 in rsi_exit_lookback:
                num_exit_signals += 1
   
            bb_entry_lookback = bb_entry[lookback_ind:i]
            if 1 in bb_entry_lookback:
                num_entry_signals += 1
            bb_exit_lookback = bb_exit[lookback_ind:i]
            if 1 in bb_exit_lookback:
                num_exit_signals += 1
     
            stc_entry_lookback = stc_entry[lookback_ind:i]
            if 1 in stc_entry_lookback:
                num_entry_signals += 1
            stc_exit_lookback = stc_exit[lookback_ind:i]
            if 1 in stc_exit_lookback:
                num_exit_signals += 1   
                
        #  Verify Entry indicators
        if hold == 0 and num_entry_signals >= required_entry_signals:
            last_entry_price = current_price
            entry_prices.append(current_price)
            exit_prices.append(np.nan)
            hold = 1
        #  Exit strategy
        elif hold == 1 and num_exit_signals >= required_exit_signals:
            entry_prices.append(np.nan)
            exit_prices.append(current_price)
            hold = 0
        else:
            #  Neither entry nor exit
            entry_prices.append(np.nan)
            exit_prices.append(np.nan)

    return entry_prices, exit_prices

In [742]:
def plot_graph(symbol, df, entry_prices, exit_prices):
    fig = make_subplots(rows=3, cols=1, subplot_titles=['Close + BB','RSI','STC'])

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['Close'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    #  Plot bollinger bands
    bb_high = df['BB_high']
    bb_mid = df['BB_mid']
    bb_low = df['BB_low']
    fig.add_trace(go.Line(x = df.index, y = bb_high, line=dict(color="orange", width=1), name="BB High"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = bb_mid, line=dict(color="#ffd866", width=1), name="BB Mid"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = bb_low, line=dict(color="orange", width=1), name="BB Low"), row = 1, col = 1)
    
    #  Plot RSI
    fig.add_trace(go.Line(x = df.index, y = df['RSI'], line=dict(color="blue", width=1), name="RSI"), row = 2, col = 1)

    #  Plot STC
    fig.add_trace(go.Line(x = df.index, y = df['STC'], line=dict(color="blue", width=1), name="STC"), row = 3, col = 1)

    #  Add buy and sell indicators
    fig.add_trace(go.Scatter(x=df.index, y=entry_prices, marker_symbol="arrow-up", marker=dict(
        color='green',
    ),mode='markers',name='Buy'))
    fig.add_trace(go.Scatter(x=df.index, y=exit_prices, marker_symbol="arrow-down", marker=dict(
        color='red'
    ),mode='markers',name='Sell'))
    
    fig.update_layout(
        title={'text':f"{symbol} with BB-RSI-STC", 'x':0.5},
        autosize=False,
        width=800,height=800)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()

In [743]:
#  Assuming reinvesting the proceeds
def calculate_profit(start_investment, entry_prices, exit_prices):
    hold = 0
    profit = 0
    available_funds = start_investment
    cost = 0
    num_stocks = 0
    for i in range(len(entry_prices)):
        current_entry_price = entry_prices[i]
        current_exit_price = exit_prices[i]

        if not math.isnan(current_entry_price) and hold == 0:
            num_stocks = available_funds / current_entry_price
            cost = num_stocks * current_entry_price
            hold = 1

        elif hold == 1 and not math.isnan(current_exit_price):
            hold = 0
            proceeds = num_stocks * current_exit_price
            profit += proceeds - cost

    return math.trunc(profit)

In [744]:
# MAIN
symbol = 'ETH-USD'
start_investment = 1000
df = load_data_file(symbol)
df = df.tail(1440)
df = calculate_tis(df)
df = calculate_signals(df)

entry_prices, exit_prices = execute_strategy(df)
profit = calculate_profit(start_investment, entry_prices, exit_prices)
trades = np.array(exit_prices)
num_trades = len(trades[~np.isnan(trades)])
print(f"Number of trades: {num_trades}")
print(f"Profit with start_investment of ${start_investment}: ${profit}")
plot_graph(symbol, df, entry_prices, exit_prices)

Number of trades: 6
Profit with start_investment of $1000: $51


C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


